<a href="https://colab.research.google.com/github/Filippo-Tombari/PdeGraph/blob/main/Dataset_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import os
from IPython.display import clear_output
import torch
from torch._C import dtype
import h5py
import pickle

In [ ]:
def readh5(filename):
  with h5py.File(filename, "r") as f:
      # List all groups
      print("Keys: %s" % f.keys())
      a_group_key = list(f.keys())[0]

      # Get the data
      data = list(f[a_group_key])
  return data

In [ ]:
#Reshape data to obtain the shape necessary for the network
def reshape_data(data, mode = 'test'):
  if(mode == 'train'):
    old_len = np.shape(data)[0]
    data = np.delete(data, range(old_len-8239,old_len))
  
  return np.reshape(data, (int(np.shape(data)[0]/2),2), order = 'F')
#Convert data from P2 discretization to P1, keeping only the vertices of the mesh
def P2toP1(data, mode = 'test'):
  data = reshape_data(data, mode)
  new_data = data[(nodes[None,:] == vertices[:,None]).all(-1).any(0)]
  return torch.from_numpy(new_data) 


# Training set


In [ ]:
!unzip /content/drive/MyDrive/tesi/obstacle/Archivio_train.zip #complete training set archive

Archive:  /content/drive/MyDrive/tesi/obstacle/Archivio_test.zip
  inflating: S_NS_test_21_end_hdf5.h5  
  inflating: params_NS_test_21_end.mat  
  inflating: params_NS_test_21_1.mat  
  inflating: S_NS_test_21_1_hdf5.h5  
  inflating: S_NS_test_21_15_hdf5.h5  
  inflating: params_NS_test_21_15.mat  


In [ ]:
#Write the training filename 
train_filename = "/content/S_NS_train_21_full_hdf5.h5" 
train_data = readh5(filename)

In [ ]:
#choose a training rollout based on a value of the parameter
train_set = train_data[400*10:400*11] # a rollout is composed of 400 time steps

In [ ]:
# create a save in a file the new training set
os.chdir('/content/drive/MyDrive/tesi/obstacle') #change directory according to the one that you want to use for saving the dataset
train_set_new = [P2toP1(data,'train') for data in train_set]
with open("train_set.pkl", "wb") as fp:   #Pickling
  pickle.dump(train_set_new, fp)

# Test set

In [ ]:
!unzip /content/drive/MyDrive/tesi/obstacle/Archivio_test.zip #complete test set archive

In [ ]:
#Write the training filename that you want to use
test_filename = "/content/S_NS_test_21_15_hdf5.h5"
test_data = readh5(filename)

In [ ]:
# create a save in a file the new test set
os.chdir('/content/drive/MyDrive/tesi/obstacle')#change directory according to the one that you want to use for saving the dataset
test_set = [P2toP1(data) for data in test_data]
with open("test_set.pkl", "wb") as fp:   #Pickling
  pickle.dump(test_set, fp)